In [2]:
#Imports
from datetime import datetime
import fsspec
#need to have google-api-python-client, google-cloud-storage, and gcloud
#installed
from google.cloud import storage
import numpy as np
import os
import pandas as pd
#need to have netcdf4 and h5netcdf installed
import xarray as xr

In [3]:
#Method to parse hurricane best track files.
#Input: Best track URL
#Output: Dataframe containing best track data
def read_best_track(bt_url):
    #Array to store hurricane location data
    hurr_info_array = []
    #Open best track file in fsspec
    with fsspec.open(bt_url) as fobj:
        #Loop through hurricanes
        while True:
            #Data includes a combination of timestamp data and overall hurricane
            #data. 
            #File format: https://www.aoml.noaa.gov/hrd/hurdat/hurdat2-format.pdf

            hurr_info = fobj.readline().decode()
            #print(hurr_info)

            #If the end of the file is reached readline() returns an empty 
            #string.
            if not hurr_info:
                break
        
            hurr_code, hurr_name, n_lines = hurr_info.replace(" ", "").split(",")[:-1]
            #Loop through timestamps within each hurricane
            for row in range(int(n_lines)):
                timestamp_info = fobj.readline().decode()
                ts_day, ts_min, record_identifier, sys_status, lat_hem, lon_hem,\
                max_sust, min_pres, rad34_ne, rad34_se, rad34_sw, rad34_nw,\
                rad50_ne, rad50_se, rad50_sw, rad50_nw, rad64_ne, rad64_se,\
                rad64_sw, rad64_nw, rad_mw = timestamp_info.replace(" ", "").split(",")
                #Data we want to store

                #parse timestamps and lat/lon
                ts_full = datetime.strptime((ts_day+ts_min),"%Y%m%d%H%M")
                
                lat_abs = float(lat_hem[:-1])
                if lat_hem[-1] == 'N':
                    lat = lat_abs
                else:
                    lat = -lat_abs

                lon_abs = float(lon_hem[:-1])
                if lon_hem[-1] == 'E':
                    lon = lon_abs
                else:
                    lon = -lon_abs

                timestamp_info_array = [hurr_code, hurr_name, ts_full, 
                                        lat_abs, lon_abs, max_sust, min_pres,
                                        rad_mw]
                hurr_info_array.append(timestamp_info_array)
    
    #Convert hurr_info_array into a dataframe
    hurr_info_df = pd.DataFrame(hurr_info_array,
                                 columns = ['Hurricane Code', 'Hurricane Name',
                                            'Timestamp', 'Latitude',
                                            'Longitude',
                                            'Maximum Sustained Winds', 
                                            'Minimum Pressure', 
                                            'Radius of Maximum Winds'])
    
    return hurr_info_df

#Import hurricane track data
atl_best_track_url = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2024-040425.txt'
atl_best_track_df = read_best_track(atl_best_track_url)

ne_pac_best_track_url = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-nepac-1949-2024-031725.txt'
ne_pac_best_track_df = read_best_track(ne_pac_best_track_url)

In [ ]:
#Reading in GLM data (didn't get very far on this)
glm_url_base_1 =  'https://storage.cloud.google.com/gcp-public-data-goes-'
glm_url_base_2 = '/GLM-L2-LCFA/2021/001/00/OR_GLM-L2-LCFA/'
#G16_s20210010000000_e20210010000205_c20210010000222.nc

#Location where your default Google credentials are stored (may need to edit)
google_creds_path = '~/.config/gcloud/application_default_credentials.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_creds_path

#Get GLM file URLs from Google Cloud bucket
client = storage.Client()
print(client.list_blobs('https://storage.cloud.google.com/gcp-public-data-goes-16/GLM-L2-LCFA/2021/001/00/'))

#To finish later

DefaultCredentialsError: File ~/.config/gcloud/application_default_credentials.json was not found.

In [ ]:
#Initial exploration of data from one file (downloaded non-programmatically)
curr_filename = '/Users/kylebretherton/Downloads/GLM-L2-LCFA_2021_001_00_OR_GLM-L2-LCFA_G16_s20210010000000_e20210010000205_c20210010000222.nc'

ds = xr.open_dataset(curr_filename)#[:, :, -6:6, -96:84, :, :, :, :, :, :, :, :, :, :, :, :, :, :, :, :, :]

center_lat = 6
center_lon = -74
box_size = 6

mask_lat = (ds['group_lat'] >= (center_lat - box_size)) & (ds['group_lat'] <= (center_lat + box_size))
mask_lon = (ds['group_lon'] >= (center_lon - box_size)) & (ds['group_lon'] <= (center_lon + box_size))
combined_mask = mask_lat & mask_lon

ds_sliced = ds.where(combined_mask, drop=True)
sliced_data = ds_sliced[['group_lat', 'group_lon']].values
sliced_data

<bound method Mapping.values of <xarray.Dataset> Size: 248B
Dimensions:                (number_of_groups: 10)
Coordinates:
    group_lat              (number_of_groups) float32 40B 6.333 6.333 ... 6.333
    group_lon              (number_of_groups) float32 40B -75.18 ... -75.18
    group_id               (number_of_groups) uint32 40B 396560068 ... 396560078
    group_time_offset      (number_of_groups) datetime64[ns] 80B 2021-01-01T0...
    group_parent_flash_id  (number_of_groups) uint16 20B 52793 52793 ... 52793
    product_time           datetime64[ns] 8B 2021-01-01
    lightning_wavelength   float32 4B 777.4
    group_time_threshold   float32 4B 0.0
    flash_time_threshold   float32 4B 3.33
    lat_field_of_view      float32 4B 0.0
    lon_field_of_view      float32 4B -75.0
Dimensions without coordinates: number_of_groups
Data variables:
    *empty*
Attributes: (12/29)
    production_site:           WCDAS
    featureType:               point
    dataset_name:              OR_GLM-

In [ ]:
#Finding number of best-track rows within our time range (about 1800 in the N
#Atlantic Basin, of which Ian makes up 40. Similar number in the NE Pacific)
start_time = datetime(2021, 1, 1, 0, 0, 0)
end_time = datetime(2023, 12, 31, 23, 59, 59)
date_mask = (atl_best_track_df.Timestamp > start_time) & (atl_best_track_df.Timestamp < end_time)
dates = atl_best_track_df.index[date_mask]
dates

Index([52901, 52902, 52903, 52904, 52905, 52906, 52907, 52908, 52909, 52910,
       ...
       54737, 54738, 54739, 54740, 54741, 54742, 54743, 54744, 54745, 54746],
      dtype='int64', length=1846)